# Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score,confusion_matrix, precision_score, f1_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier
from sklearn import svm
import time
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
import zipfile
import joblib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = !kaggle datasets download -d shayanfazeli/heartbeat

In [ ]:
with zipfile.ZipFile('heartbeat.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

# Load the data

In [ ]:
train_data = pd.read_csv(r"/content/data/mitbih_train.csv")
test_data = pd.read_csv(r"/content/data/mitbih_test.csv")

train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

del train_data, test_data

train_df.columns = [f'Feature {i + 1}' for i in range(len(train_df.columns) - 1)] + ['Target']
test_df.columns = [f'Feature {i + 1}' for i in range(len(test_df.columns) - 1)] + ['Target']
train_df

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,...,Feature 179,Feature 180,Feature 181,Feature 182,Feature 183,Feature 184,Feature 185,Feature 186,Feature 187,Target
0,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.927461,1.000000,0.626943,0.193437,0.094991,0.072539,0.043178,0.053541,0.093264,0.189983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87548,0.807018,0.494737,0.536842,0.529825,0.491228,0.484211,0.456140,0.396491,0.284211,0.136842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87549,0.718333,0.605000,0.486667,0.361667,0.231667,0.120000,0.051667,0.001667,0.000000,0.013333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87550,0.906122,0.624490,0.595918,0.575510,0.530612,0.481633,0.444898,0.387755,0.322449,0.191837,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87551,0.858228,0.645570,0.845570,0.248101,0.167089,0.131646,0.121519,0.121519,0.118987,0.103797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


# Data analysis

In [ ]:
train_df.info(), test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87553 entries, 0 to 87552
Columns: 188 entries, Feature 1 to Target
dtypes: float64(188)
memory usage: 125.6 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21891 entries, 0 to 21890
Columns: 188 entries, Feature 1 to Target
dtypes: float64(188)
memory usage: 31.4 MB


(None, None)

In [ ]:
train_df.isnull().sum(), test_df.isnull().sum()

(Feature 1      0
 Feature 2      0
 Feature 3      0
 Feature 4      0
 Feature 5      0
               ..
 Feature 184    0
 Feature 185    0
 Feature 186    0
 Feature 187    0
 Target         0
 Length: 188, dtype: int64,
 Feature 1      0
 Feature 2      0
 Feature 3      0
 Feature 4      0
 Feature 5      0
               ..
 Feature 184    0
 Feature 185    0
 Feature 186    0
 Feature 187    0
 Target         0
 Length: 188, dtype: int64)

In [ ]:
train_df['Target']=train_df['Target'].astype(int)
train_df['Target'].nunique()

5

In [ ]:
train_df['Target'].value_counts()

,count
Target,
0,72470
4,6431
2,5788
1,2223
3,641


# **Stratage 1 : Apply Multi classifier on 5 classes**

# Convert data to binary classification dataset

In [ ]:
train_df_new = train_df.copy()
test_df_new = test_df.copy()

train_df_new['Binary_Target'] = train_df['Target'].apply(lambda x: 0 if x == 0 else 1)
test_df_new['Binary_Target'] = test_df['Target'].apply(lambda x: 0 if x == 0 else 1)

train_df_new['Binary_Target'].value_counts(),test_df_new['Binary_Target'].value_counts()

(Binary_Target
 0    72470
 1    15083
 Name: count, dtype: int64,
 Binary_Target
 0    18117
 1     3774
 Name: count, dtype: int64)

# Split data and apply feature reduction

In [ ]:
X_train = train_df_new.drop(columns=['Binary_Target'])
y_train = train_df_new['Binary_Target']
X_test = test_df_new.drop(columns=['Binary_Target'])
y_test = test_df_new['Binary_Target']

pca = PCA(n_components=0.99)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

joblib.dump(pca, 'pca_transform.pkl')

num_components = X_train_pca.shape[1]
print(f"Number of principal components to represent 99% of data: {num_components}")

print(X_train_pca[:5])

Number of principal components to represent 99% of data: 66
[[-1.52613972e+00 -1.10147187e-01  6.01091037e-01 -5.47300581e-01
   4.32878385e-01 -1.59621245e-02  3.52399406e-01  7.64144984e-02
  -1.55255899e-01  8.32902647e-02 -4.34076050e-01 -3.34128395e-01
   1.25118467e-01  8.12535795e-01 -3.69961806e-01  3.00973116e-01
   3.87178062e-01 -3.68464787e-01  1.59436679e-01 -2.15309064e-01
   2.23890768e-01 -5.78175803e-02 -3.11541582e-02  9.91323267e-02
   3.89400575e-01 -1.52223448e-02 -4.94298904e-01 -2.15185291e-01
   3.85123638e-02 -2.04652301e-01  2.31975265e-02 -1.26092490e-01
   2.01067297e-02 -6.25917577e-02 -1.72743708e-01  2.44750552e-01
  -1.39983118e-01 -5.61698109e-02  8.45788685e-02 -1.78382182e-01
   5.93912731e-02  1.23845026e-01 -5.67959090e-02  5.41000949e-02
   9.10782274e-02 -1.17390397e-02 -9.61933511e-02 -3.29084723e-02
   1.15787236e-01  6.67081158e-02  2.99743126e-02  1.57059329e-02
   4.32822245e-02  4.87716482e-02 -3.09372136e-02 -2.46699776e-02
   2.44593970e-0

# Binary classifier between normal and abnormal

In [ ]:
classifier = RandomForestClassifier(n_estimators=200, max_depth=None, class_weight={0: 1, 1: 2})
classifier.fit(X_train_pca, y_train)

y_pred_train = classifier.predict(X_train_pca)
y_pred_test = classifier.predict(X_test_pca)

joblib.dump(classifier, 'Binary classifier_random_forest_model.pkl')

# Evaluate binary classifier
print("Binary Classifier Evaluation:")
print(classification_report(y_test, y_pred_test))
print("Confusion Matrix (Binary Classifier):")
print(confusion_matrix(y_test, y_pred_test))

Binary Classifier Evaluation:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18117
           1       1.00      0.99      1.00      3774

    accuracy                           1.00     21891
   macro avg       1.00      1.00      1.00     21891
weighted avg       1.00      1.00      1.00     21891

Confusion Matrix (Binary Classifier):
[[18116     1]
 [   22  3752]]


# Prepare sub-dataset for abnormal data (4 sub-classes)

In [ ]:
indices_sub_test = np.where(y_pred_test != 0)[0]
indices_sub_train = np.where(y_train != 0)[0]

X_train_sub = X_train.iloc[indices_sub_train]
y_train_sub = train_df.iloc[indices_sub_train]['Target']

X_test_sub = X_test.iloc[indices_sub_test]
y_test_sub = test_df.iloc[indices_sub_test]['Target']

X_train_sub = X_train_sub[y_train_sub != 0]
y_train_sub = y_train_sub[y_train_sub != 0]
X_test_sub = X_test_sub[y_test_sub != 0]
y_test_sub = y_test_sub[y_test_sub != 0]

# Feature reduction

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train_sub)
X_train_sub_scaled = scaler.transform(X_train_sub)
X_test_sub_scaled = scaler.transform(X_test_sub)

pca = PCA(n_components=0.99)
X_train_sub_pca = pca.fit_transform(X_train_sub_scaled)
X_test_sub_pca = pca.transform(X_test_sub_scaled)

num_components = X_train_sub_pca.shape[1]
print(f"Number of principal components to represent 99% of data: {num_components}")

Number of principal components to represent 99% of data: 75


# Handle class imbalance for sub-classifier

In [ ]:
smote = SMOTE(random_state=42)
X_train_sub_balanced, y_train_sub_balanced = smote.fit_resample(X_train_sub_pca, y_train_sub)
X_train_sub_balanced,y_train_sub_balanced

(array([[-3.49029084e+00, -2.19750814e+00, -2.85708742e+00, ...,
          9.98523861e-02, -1.61410870e-01, -2.61915907e-01],
        [ 8.86553429e+00,  2.82548769e+01, -5.22682810e+00, ...,
         -4.03234985e-02,  5.99867935e-03, -6.67275812e-02],
        [ 3.43491216e-01, -2.92219610e+00, -3.19566077e+00, ...,
          2.88252222e-02, -1.67688220e-01, -3.05732488e-01],
        ...,
        [-9.97137326e+00, -1.29602000e+00, -2.73262019e+00, ...,
         -3.70086624e-02, -1.23902284e-01, -1.71935138e-01],
        [-1.03706429e+01, -8.39849398e-01, -2.12813711e-01, ...,
         -1.45036063e-01, -1.50847628e-01, -4.55311842e-02],
        [-5.70671674e+00, -2.23476576e+00, -4.77672153e+00, ...,
         -1.65258333e-01, -2.29942043e-01, -1.93290490e-01]]),
 0        1
 1        1
 2        1
 3        1
 4        1
         ..
 25719    3
 25720    3
 25721    3
 25722    3
 25723    3
 Name: Target, Length: 25724, dtype: int64)

# Train sub-classifier on 4 abnormal classes

In [ ]:
sub_classifier = RandomForestClassifier(class_weight='balanced')
sub_classifier.fit(X_train_sub_balanced, y_train_sub_balanced)

y_pred_sub_test = sub_classifier.predict(X_test_sub_pca)

joblib.dump(classifier, 'sub-classifier_random_forest_model.pkl')

print("\nSub-classifier Evaluation:")
print(classification_report(y_test_sub, y_pred_sub_test))
print("Confusion Matrix (Sub-classifier):")
print(confusion_matrix(y_test_sub, y_pred_sub_test))


Sub-classifier Evaluation:
              precision    recall  f1-score   support

         1.0       0.98      0.93      0.96       539
         2.0       0.95      0.98      0.97      1443
         3.0       0.90      0.92      0.91       162
         4.0       1.00      0.98      0.99      1608

    accuracy                           0.97      3752
   macro avg       0.96      0.95      0.96      3752
weighted avg       0.97      0.97      0.97      3752

Confusion Matrix (Sub-classifier):
[[ 501   38    0    0]
 [   7 1420   16    0]
 [   1   12  149    0]
 [   0   27    0 1581]]


# Combine binary classifier and sub-classifier predictions

In [ ]:
final_predictions = y_pred_test.copy()

print(f"Length of indices_sub_test: {len(indices_sub_test)}")
print(f"Length of y_pred_sub_test: {len(y_pred_sub_test)}")

if len(indices_sub_test) == len(y_pred_sub_test):
    final_predictions[indices_sub_test] = y_pred_sub_test
else:
    print("Warning: Length mismatch between indices_sub_test and y_pred_sub_test.")
    min_length = min(len(indices_sub_test), len(y_pred_sub_test))
    final_predictions[indices_sub_test[:min_length]] = y_pred_sub_test[:min_length]

y_test_true = test_df['Target']

print("\nFinal Model Evaluation:")
print(classification_report(y_test_true, final_predictions))
print("Confusion Matrix (Final Model):")
print(confusion_matrix(y_test_true, final_predictions))

Length of indices_sub_test: 3753
Length of y_pred_sub_test: 3752

Final Model Evaluation:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     18117
         1.0       0.98      0.90      0.94       556
         2.0       0.95      0.98      0.96      1448
         3.0       0.90      0.91      0.91       162
         4.0       1.00      0.98      0.99      1608

    accuracy                           0.99     21891
   macro avg       0.96      0.96      0.96     21891
weighted avg       0.99      0.99      0.99     21891

Confusion Matrix (Final Model):
[[18116     1     0     0     0]
 [   17   500    39     0     0]
 [    5     7  1419    17     0]
 [    0     1    12   148     1]
 [    0     1    27     0  1580]]


 # **Stratage 2 : Apply single classifier on 5 classes**

# Split original data and apply feature reduction

In [ ]:
X_train = train_df.drop(columns=['Target'])
y_train = train_df['Target']
X_test = test_df.drop(columns=['Target'])
y_test = test_df['Target']

pca = PCA(n_components=0.99)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

num_components = X_train_pca.shape[1]
print(f"Number of principal components to represent 99% of data: {num_components}")

print(X_train_pca[:5])

Number of principal components to represent 99% of data: 69
[[-1.43690148e+00  6.04470657e-01 -5.69384516e-01 -4.13983099e-01
  -1.94281190e-01 -3.80064480e-01  7.90888493e-02 -1.66082698e-02
  -1.57491673e-01 -4.99966115e-01  4.86823525e-02 -2.20451798e-01
   8.07113829e-01 -2.19038767e-01 -6.16276092e-02 -6.04224348e-01
  -4.14971532e-02 -3.63421356e-01 -2.13966063e-01  2.22595931e-01
  -6.00413637e-02 -2.61408216e-02  1.00116957e-01  3.89872645e-01
  -8.70045330e-03 -4.87645498e-01 -2.10611896e-01 -5.36522812e-02
  -2.23976615e-01  1.70277009e-02  1.09437320e-01  6.72296916e-02
  -5.06268503e-02 -1.58614723e-01  2.47903778e-01 -1.51284748e-01
  -6.26336528e-02  8.37341199e-02 -1.77732150e-01  3.99652609e-02
   1.28455818e-01 -5.71524592e-02  5.29645149e-02  8.66885230e-02
   1.34563071e-02 -9.29615989e-02 -5.07336576e-02  1.08594479e-01
   7.43373748e-02  3.48211040e-02  1.41664708e-02  4.52004875e-02
   4.74414350e-02 -3.37716859e-02 -2.58754503e-02  2.33129153e-02
  -6.92899227e-0

In [ ]:
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train_pca, y_train)

X_train, y_train

(array([[-1.43690148e+00,  6.04470657e-01, -5.69384516e-01, ...,
         -1.99322189e-02,  2.42256467e-02, -1.51218553e-02],
        [-1.72058876e+00, -7.51246112e-02, -2.45866647e-01, ...,
          3.91171712e-02, -6.50411024e-02, -3.30869341e-02],
        [-1.50937052e+00, -4.47572579e-02,  3.49775642e-01, ...,
          2.56746905e-02, -2.09032984e-02, -1.76337145e-02],
        ...,
        [ 1.25294457e+00, -1.41267537e-01,  7.43625819e-01, ...,
         -3.47653607e-02, -2.42910955e-02, -5.94618363e-02],
        [ 2.76877134e+00,  2.79654669e-01,  8.96367959e-01, ...,
         -7.43573247e-03,  7.58750953e-02,  3.09548076e-03],
        [ 3.27845758e+00, -4.67992850e-01,  8.23447607e-01, ...,
          4.64438983e-02,  1.44695928e-02, -2.36959324e-02]]),
 0         0
 1         0
 2         0
 3         0
 4         0
          ..
 362345    4
 362346    4
 362347    4
 362348    4
 362349    4
 Name: Target, Length: 362350, dtype: int64)

# Apply Singe classifier

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

y_pred_train = knn_model.predict(X_train)
y_pred_test = knn_model.predict(X_test_pca)

train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

joblib.dump(classifier, 'knn_model.pkl')

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

print("\nClassification Report (Test Data):")
print(classification_report(y_test, y_pred_test))

print("Confusion Matrix (Test Data):")
print(confusion_matrix(y_test, y_pred_test))

Training Accuracy: 0.9944
Test Accuracy: 0.9574

Classification Report (Test Data):
              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98     18117
         1.0       0.53      0.82      0.64       556
         2.0       0.88      0.94      0.91      1448
         3.0       0.49      0.83      0.62       162
         4.0       0.96      0.98      0.97      1608

    accuracy                           0.96     21891
   macro avg       0.77      0.91      0.82     21891
weighted avg       0.97      0.96      0.96     21891

Confusion Matrix (Test Data):
[[17436   386   144    97    54]
 [   75   457    17     5     2]
 [   31    17  1361    30     9]
 [   12     2    14   134     0]
 [   19     6     8     5  1570]]


In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred_train_rf = rf_classifier.predict(X_train)
y_pred_test_rf = rf_classifier.predict(X_test_pca)

train_accuracy_rf = accuracy_score(y_train, y_pred_train_rf)
test_accuracy_rf = accuracy_score(y_test, y_pred_test_rf)

joblib.dump(classifier, 'rf_classifier.pkl')

print(f"Random Forest - Training Accuracy: {train_accuracy_rf:.4f}")
print(f"Random Forest - Test Accuracy: {test_accuracy_rf:.4f}")

print("\nRandom Forest - Classification Report (Test Data):")
print(classification_report(y_test, y_pred_test_rf))

print("Confusion Matrix (Test Data):")
print(confusion_matrix(y_test, y_pred_test_rf))

Random Forest - Training Accuracy: 1.0000
Random Forest - Test Accuracy: 0.9781

Random Forest - Classification Report (Test Data):
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99     18117
         1.0       0.87      0.73      0.79       556
         2.0       0.95      0.92      0.94      1448
         3.0       0.79      0.73      0.76       162
         4.0       1.00      0.96      0.98      1608

    accuracy                           0.98     21891
   macro avg       0.92      0.87      0.89     21891
weighted avg       0.98      0.98      0.98     21891

Confusion Matrix (Test Data):
[[18010    56    37    12     2]
 [  143   405     7     0     1]
 [   93     1  1334    19     1]
 [   29     0    15   118     0]
 [   53     2     8     0  1545]]
